# Fruit recognition

## Chargement des modules

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as im
import sklearn as sk

## Chargement des données

In [6]:
tabImg=pd.DataFrame()
import glob
root_dir="train/"
for filename in glob.iglob(root_dir+'**/*.jpg',recursive=True):
    img=im.imread(filename)
    img=img.reshape((1,img.shape[0]*img.shape[1]*img.shape[2]))
    
    fruit_name=filename[6:].split('/')[0]
    
    new_row={'Fruit':fruit_name,'Image':img}
    tabImg=tabImg.append(new_row,ignore_index=True)

In [7]:
tabImg.shape

(16854, 2)

In [8]:
tabImg.to_csv('train.csv',index=False)

In [ ]:
tabImg.head(5)